In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

%precision 3
%matplotlib inline

In [ ]:
df = pd.read_csv('./data/ch12_scores_reg.csv')
n = len(df)
print(n)
df

In [ ]:
x = np.array(df['小テスト'])
y = np.array(df['期末テスト'])
p = 1

In [ ]:
x

In [ ]:
y

In [ ]:
poly_fit = np.polyfit(x, y, 1)
print(poly_fit)
poly_1d = np.poly1d(poly_fit)
print(poly_1d)
xs = np.linspace(x.min(), x.max())
ys = poly_1d(xs)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)

ax.set_xlabel('小テスト')
ax.set_ylabel('期末テスト')
ax.plot(xs, ys, color='gray', label=f'{poly_fit[1]:.2f}+{poly_fit[0]:.2f}x')
ax.scatter(x, y)
ax.legend()

plt.show()

In [ ]:
formula = '期末テスト ~ 小テスト'
result = smf.ols(formula, df).fit()
result.summary()

In [ ]:
X = np.array([np.ones_like(x), x]).T
X

In [ ]:
beta0_hat, beta1_hat = np.linalg.lstsq(X, y)[0]
beta0_hat, beta1_hat

In [ ]:
y_hat = beta0_hat + beta1_hat * x
eps_hat = y - y_hat
eps_hat

In [ ]:
s_var = np.var(eps_hat, ddof=p+1)
s_var

In [ ]:
C0, C1 = np.diag(np.linalg.pinv(np.dot(X.T, X)))

In [ ]:
np.sqrt(s_var * C0), np.sqrt(s_var * C1)

In [ ]:
rv = stats.t(n-2)

lcl = beta0_hat - rv.isf(0.025) * np.sqrt(s_var * C0)
ucl = beta0_hat - rv.isf(0.975) * np.sqrt(s_var * C0)

lcl, ucl

In [ ]:
rv = stats.t(n-2)

lcl = beta1_hat - rv.isf(0.025) * np.sqrt(s_var * C1)
ucl = beta1_hat - rv.isf(0.975) * np.sqrt(s_var * C1)

lcl, ucl

In [ ]:
t = beta1_hat / np.sqrt(s_var * C1)
t

In [ ]:
(1 - rv.cdf(t)) * 2

In [ ]:
t = beta0_hat / np.sqrt(s_var * C0)
t

In [ ]:
(1 - rv.cdf(t)) * 2

In [ ]:
formula = '期末テスト ~ 小テスト + 睡眠時間'
result = smf.ols(formula, df).fit()
result.summary()

In [ ]:
x1 = np.array(df['小テスト'])
x2 = np.array(df['睡眠時間'])
y = np.array(df['期末テスト'])
p = 2

In [ ]:
X = np.array([np.ones_like(x1), x1, x2]).T
X

In [ ]:
beta0_hat, beta1_hat, beta2_hat = np.linalg.lstsq(X, y)[0]
beta0_hat, beta1_hat, beta2_hat

In [ ]:
y_hat = beta0_hat + beta1_hat * x1 + beta2_hat * x2
eps_hat = y - y_hat
eps_hat

In [ ]:
s_var = np.var(eps_hat, ddof=p+1)
s_var

In [ ]:
s_var = np.sum(eps_hat ** 2) / (n - p - 1)
s_var

In [ ]:
C0, C1, C2 = np.diag(np.linalg.pinv(np.dot(X.T, X)))

In [ ]:
rv = stats.t(n-p-1)

lcl = beta2_hat - rv.isf(0.025) * np.sqrt(s_var * C2)
ucl = beta2_hat - rv.isf(0.975) * np.sqrt(s_var * C2)

lcl, ucl

In [ ]:
formula = '期末テスト ~ 小テスト + 睡眠時間 + 通学方法'
result = smf.ols(formula, df).fit()
result.summary()

In [ ]:
x = np.array(df['小テスト'])
y = np.array(df['期末テスト'])
p = 1

In [ ]:
formula = '期末テスト ~ 小テスト'
result = smf.ols(formula, df).fit()
result.summary()

In [ ]:
result.fittedvalues

In [ ]:
y_hat = np.array(result.fittedvalues)
y_hat

In [ ]:
result.resid

In [ ]:
eps_hat = np.array(result.resid)
eps_hat

In [ ]:
np.sum(eps_hat ** 2)

In [ ]:
total_var = np.sum((y - np.mean(y))**2)
exp_var = np.sum((y_hat - np.mean(y))**2)
unexp_var = np.sum(eps_hat ** 2)
total_var, exp_var, unexp_var

In [ ]:
total_var, exp_var + unexp_var

In [ ]:
exp_var / total_var

In [ ]:
np.corrcoef(x, y)[0, 1] ** 2

In [ ]:
1 - (unexp_var / (n - p - 1)) / (total_var / (n - 1))

In [ ]:
f = (exp_var / p) / (unexp_var / (n - p - 1))
f

In [ ]:
rv = stats.f(p, n-p-1)
1 - rv.cdf(f)

In [ ]:
prob = 0.3
coin_result = [0, 1, 0, 0, 1]

In [ ]:
rv = stats.bernoulli(prob)
rv.pmf(coin_result)

In [ ]:
L = np.prod(rv.pmf(coin_result))
L

In [ ]:
ps = np.linspace(0, 1, 100)
Ls = [np.prod(stats.bernoulli(prob).pmf(coin_result)) for prob in ps]

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

ax.plot(ps, Ls, label='尤度関数', color='gray')
ax.legend()

plt.show()

In [ ]:
prob = 0.4
rv = stats.bernoulli(prob)
mll = np.sum(np.log(rv.pmf([0, 1, 0, 0, 1])))
mll

In [ ]:
rv = stats.norm(y_hat, np.sqrt(unexp_var / n))
mll = np.sum(np.log(rv.pdf(y)))
mll

In [ ]:
aic = -2 * mll + 2 * (p + 1)
aic

In [ ]:
bic = -2 * mll + np.log(n) * (p + 1)
bic

In [ ]:
formula = '期末テスト ~ 小テスト + 睡眠時間'
result = smf.ols(formula, df).fit()
result.summary()

In [ ]:
eps_hat = np.array(result.resid)

In [ ]:
stats.skew(eps_hat)

In [ ]:
stats.kurtosis(eps_hat, fisher=False)

In [ ]:
np.sum(np.diff(eps_hat, 1) ** 2) / np.sum(eps_hat ** 2)

In [ ]:
df['中テスト'] = df['小テスト'] * 2
df

In [ ]:
formula = '期末テスト ~ 小テスト + 中テスト'
result = smf.ols(formula, df).fit()
result.summary()